# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [3]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import re
import pickle
import nltk

nltk.download("punkt")
nltk.download("stopwords")

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    make_scorer,
)
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

import warnings

warnings.simplefilter("ignore")

[nltk_data] Downloading package punkt to /home/roger/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/roger/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [41]:
# load data from database
engine = create_engine("sqlite:///disaster_response.db")
df = pd.read_sql("SELECT * FROM messages", engine)
# random sample 10% of the data. Delete this line to run analysis on full data set
df = df.sample(frac=0.1, replace=False, random_state=1)
X = df["message"]
Y = df.drop(["id", "message", "original", "genre"], axis=1)

### 2. Write a tokenization function to process your text data

In [34]:
def tokenize(text):
    """Normalize, tokenize and stem text string
    
    Args:
    text: string. String containing message for processing
       
    Returns:
    stemmed: list of strings. List containing normalized and stemmed word tokens
    """
    # Convert text to lowercase and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())

    # Tokenize words
    tokens = word_tokenize(text)

    # Stem word tokens and remove stop words
    stemmer = PorterStemmer()
    stop_words = stopwords.words("english")

    stemmed = [stemmer.stem(word) for word in tokens if word not in stop_words]

    return stemmed

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [35]:
pipeline = Pipeline(
    [
        ("vect", CountVectorizer(tokenizer=tokenize)),
        ("tfidf", TfidfTransformer()),
        ("clf", MultiOutputClassifier(RandomForestClassifier())),
    ]
)

### 4. Train pipeline
- Split data into train and test sets
-Train pipeline

In [42]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=1)
np.random.seed(17)
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                                                                        ccp_alpha=0.0,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                   

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [58]:
# Make predictions on the training data
Y_pred = pipeline.predict(X_test)

In [38]:
from sklearn.metrics import classification_report

In [39]:
def report_df(y_true, y_pred, col_names):
    assert len(y_true) == len(y_pred) and y_pred.shape[1] == len(col_names)
    report_list = []
    for i in range(len(col_names)):
        # get run classification_report for each column
        metrics = classification_report(
            y_true.iloc[:, i], y_pred[:, i], output_dict=True
        )
        # extract the "weighted avg" f1, precision, recall scores and store it as a dict
        scores_dict = metrics.get("weighted avg")
        # extract the accuracy score and update the dict
        scores_dict.update({"Accuracy": metrics.get("accuracy")})
        # update the dict with prediction class variable names
        scores_dict.update({"class_name": col_names[i]})
        # append the dict to the list
        report_list.append(scores_dict)
    # return the summarized model valuation metrics as a DataFrame
    report_df = pd.DataFrame(report_list) 
    report_df.set_index('class_name',inplace = True)
    return report_df 

In [59]:
report_df(Y_test,Y_pred, Y_test.columns)

,precision,recall,f1-score,support,Accuracy
class_name,,,,,
related,0.758486,0.786585,0.753649,656,0.786585
request,0.881235,0.881098,0.861172,656,0.881098
offer,0.987842,0.993902,0.990863,656,0.993902
aid_related,0.762354,0.763720,0.759561,656,0.763720
medical_help,0.853370,0.923780,0.887181,656,0.923780
medical_products,0.893256,0.945122,0.918457,656,0.945122
search_and_rescue,0.969745,0.984756,0.977193,656,0.984756
security,0.981791,0.990854,0.986301,656,0.990854
military,0.957773,0.978659,0.968103,656,0.978659


### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 
cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.